# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats


def import_cities_data():
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    cities["city"] = cities["Metropolitan area"].str.replace("–.*", "", regex=True)
    cities["2016 population"] = cities["Population (2016 est.)[8]"].astype(int)
    cities.drop(
        columns=["Metropolitan area", "Population (2016 est.)[8]"], inplace=True
    )
    for column in ["NHL", "NBA", "MLB", "NFL"]:
        cities[column] = cities[column].str.replace("\[.*\]", "", regex=True)
    cities.replace(to_replace=["—", ""], value=np.nan, inplace=True)
    new_york_rows = {
        "city": ["New York City", "New York City", "New York City"],
        "NFL": ["Giants", "Jets", np.nan],
        "MLB": ["Yankees", "Mets", np.nan],
        "NBA": ["Knicks", "Nets", np.nan],
        "NHL": ["Rangers", "Islanders", "Devils"],
        "2016 population": [20153634, 20153634, 20153634],
    }
    los_angeles_rows = {
        "city": ["Los Angeles", "Los Angeles"],
        "NFL": ["Rams", "Chargers"],
        "MLB": ["Dodgers", "Angels"],
        "NBA": ["Lakers", "Clippers"],
        "NHL": ["Kings", "Ducks"],
        "2016 population": [13310447, 13310447],
    }
    sf_rows = {
        "city": ["San Francisco Bay Area", "San Francisco Bay Area"],
        "NFL": ["49ers", "Raiders"],
        "MLB": ["Giants", "Athletics"],
        "NBA": ["Warriors", np.nan],
        "NHL": ["Sharks", np.nan],
        "2016 population": [6657982, 6657982],
    }
    chicago_rows = {
        "city": ["Chicago", "Chicago"],
        "NFL": ["Bears", np.nan],
        "MLB": ["Cubs", "White Sox"],
        "NBA": ["Bulls", np.nan],
        "NHL": ["Blackhawks", np.nan],
        "2016 population": [9512999, 9512999],
    }
    for dict in [new_york_rows, los_angeles_rows, sf_rows, chicago_rows]:
        new_df = pd.DataFrame(dict)
        cities = cities[cities["city"] != dict["city"][0]]
        cities = pd.concat([cities, new_df])
    return cities


def clean_team_name(team_name, city_df):
    team_name = team_name.lower().strip()
    cities_list = [city.lower().strip() for city in city_df["city"]]
    cities_list.extend(
        [
            "florida",
            "tampa bay",
            "vegas",
            "washington",
            "carolina",
            "minnesota",
            "colorado",
            "arizona",
            "san jose",
            "anaheim",
            "new jersey",
            "utah",
            "indiana",
            "golden state",
            "city",
            "new york",
            "brooklyn",
            "oakland",
            "texas",
            "san francisco",
            "new england",
            "tennessee",
        ]
    )
    for city in cities_list:
        if city in team_name:
            team_name = team_name.replace(city, "").strip()
    return team_name.title()


def join_nhl_df(nhl_df, cities_df):
    nhl_df = nhl_df[nhl_df["year"] == 2018]
    nhl_df.loc[:, "team"] = nhl_df["team"].str.replace("\*", "", regex=True)
    nhl_df.loc[:, "team"] = nhl_df["team"].apply(clean_team_name, args=(cities_df,))
    joined = pd.merge(
        left=nhl_df[["team", "year", "W", "L"]],
        right=cities_df[["city", "NHL", "2016 population"]],
        how="left",
        left_on="team",
        right_on="NHL",
    )
    joined.dropna(subset="city", inplace=True)
    for column in ["W", "L"]:
        joined[column] = joined[column].astype(float)
    joined["win_loss_ratio"] = joined.apply(
        lambda x: x["W"] / (x["W"] + x["L"]), axis="columns"
    )
    return joined


def nhl_correlation():
    # YOUR CODE HERE
    cities = import_cities_data()
    nhl_df = pd.read_csv("assets/nhl.csv")
    joined = join_nhl_df(nhl_df, cities)
    wl = list(joined.groupby("city").agg({"win_loss_ratio": "mean"})["win_loss_ratio"])
    pop = list(
        joined.groupby("city").agg({"2016 population": "mean"})["2016 population"]
    )
    # return stats.pearsonr(population_by_region, win_loss_by_region)
    r, p_value = stats.pearsonr(wl, pop)
    return r
    raise NotImplementedError()


nhl_correlation()

0.012486162921209881

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats


def join_nba_df(nba_df, cities_df):
    nba_df = nba_df.loc[nba_df["year"] == 2018]
    nba_df.loc[:, "team"] = (
        nba_df["team"].str.strip().replace(r"\b\*?\s\(.*\)$", "", regex=True)
    )
    nba_df.loc[:, "team"] = nba_df["team"].apply(clean_team_name, args=(cities_df,))
    nba_df.loc[nba_df["team"] == "76Ers", "team"] = "76ers"
    joined = pd.merge(
        left=nba_df[["team", "year", "W", "L", "W/L%"]],
        right=cities_df[["city", "NBA", "2016 population"]],
        how="left",
        left_on="team",
        right_on="NBA",
    )
    for column in ["W", "L", "W/L%"]:
        joined[column] = joined[column].astype(float)
    joined["win_loss_ratio"] = joined.apply(
        lambda x: x["W"] / (x["W"] + x["L"]), axis="columns"
    )
    return joined


def nba_correlation():
    # YOUR CODE HERE
    cities = import_cities_data()
    nba_df = pd.read_csv("assets/nba.csv")
    joined = join_nba_df(nba_df, cities)
    grouped = joined.groupby("city")
    wl = list(grouped.agg({"win_loss_ratio": "mean"})["win_loss_ratio"])
    # wl = list(grouped.agg({"W/L%": "mean"})["W/L%"])
    pop = list(grouped.agg({"2016 population": "mean"})["2016 population"])
    r, p_value = stats.pearsonr(wl, pop)
    return r
    raise NotImplementedError()


nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats


def join_mlb_df(mlb_df, cities_df):
    mlb_df = mlb_df.loc[mlb_df["year"] == 2018]
    mlb_df.loc[:, "team"] = mlb_df["team"].apply(clean_team_name, args=(cities_df,))
    joined = pd.merge(
        left=mlb_df[["team", "year", "W", "L", "W-L%"]],
        right=cities_df[["city", "2016 population", "MLB"]],
        how="left",
        left_on="team",
        right_on="MLB",
    )
    for column in ["W", "L"]:
        joined[column] = joined[column].astype(float)

    joined["win_loss_ratio"] = joined.apply(
        lambda x: x["W"] / (x["W"] + x["L"]), axis="columns"
    )
    return joined


def mlb_correlation():
    # YOUR CODE HERE
    cities = import_cities_data()
    mlb_df = pd.read_csv("assets/mlb.csv")
    joined = join_mlb_df(mlb_df, cities)
    grouped = joined.groupby("city")
    wl = list(grouped.agg({"win_loss_ratio": "mean"})["win_loss_ratio"])
    pop = list(grouped.agg({"2016 population": "mean"})["2016 population"])
    r, p_value = stats.pearsonr(wl, pop)
    return r
    raise NotImplementedError()

mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats


def join_nfl_data(nfl_df, cities_df):
    nfl_df = nfl_df.loc[nfl_df["year"] == 2018]
    nfl_df.loc[:, "team"] = (
        nfl_df["team"].str.strip().replace(r"[\*\+]", "", regex=True)
    )
    nfl_df.loc[:, "team"] = nfl_df["team"].apply(clean_team_name, args=(cities_df,))
    nfl_df.loc[nfl_df["team"] == "49Ers", "team"] = "49ers"
    joined = pd.merge(
        left=nfl_df[["team", "year", "W", "L", "W-L%"]],
        right=cities_df[["2016 population", "city", "NFL"]],
        how="left",
        left_on="team",
        right_on="NFL",
    )
    for column in ["W", "L", "W-L%"]:
        joined[column] = pd.to_numeric(joined[column], errors="coerce")

    joined["win_loss_ratio"] = joined.apply(
        lambda x: x["W"] / (x["W"] + x["L"]), axis="columns"
    )
    joined.dropna(subset="city", inplace=True)
    return joined


def nfl_correlation():
    # YOUR CODE HERE
    cities = import_cities_data()
    nfl_df = pd.read_csv("assets/nfl.csv")
    joined = join_nfl_data(nfl_df, cities)
    grouped = joined.groupby("city")
    wl = list(grouped.agg({"win_loss_ratio": "mean"})["win_loss_ratio"])
    pop = list(grouped.agg({"2016 population": "mean"})["2016 population"])
    r, p_value = stats.pearsonr(wl, pop)
    return r
    raise NotImplementedError()


nfl_correlation()

0.004922112149349434

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [51]:
import pandas as pd
import numpy as np
import scipy.stats as stats


def sports_team_performance():
    # YOUR CODE HERE
    mlb_df = pd.read_csv("assets/mlb.csv")
    nhl_df = pd.read_csv("assets/nhl.csv")
    nba_df = pd.read_csv("assets/nba.csv")
    nfl_df = pd.read_csv("assets/nfl.csv")
    cities = import_cities_data()
    joined_mlb = join_mlb_df(mlb_df, cities)
    joined_nhl = join_nhl_df(nhl_df, cities)
    joined_nba = join_nba_df(nba_df, cities)
    joined_nfl = join_nfl_data(nfl_df, cities)
    all_joined = {
        "MLB": joined_mlb,
        "NHL": joined_nhl,
        "NBA": joined_nba,
        "NFL": joined_nfl,
    }
    p_values = pd.DataFrame(
        {k: np.nan for k in all_joined.keys()}, index=all_joined.keys()
    )
    for sport_1, df_1 in all_joined.items():
        for sport_2, df_2 in all_joined.items():
            if sport_1 != sport_2:
                joined_df = pd.merge(
                    df_1,
                    df_2,
                    how="inner",
                    on="city",
                    suffixes=(f"_{sport_1}", f"_{sport_2}"),
                )
                grouped_df = joined_df.groupby("city")
                aggregated = grouped_df[
                    [f"win_loss_ratio_{sport_1}", f"win_loss_ratio_{sport_2}"]
                ].agg("mean")
                if len(aggregated) > 1:
                    t, p = stats.ttest_rel(
                        aggregated[f"win_loss_ratio_{sport_1}"],
                        aggregated[f"win_loss_ratio_{sport_2}"],
                    )
                    p_values.loc[sport_1, sport_2] = p
                # if sport_1 == "NBA" and sport_2 == "NFL":
                #     return aggregated
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan

    assert (
        abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2
    ), "The NBA-NHL p-value should be around 0.02"
    assert (
        abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2
    ), "The MLB-NFL p-value should be around 0.80"
    return p_values
    raise NotImplementedError()


sports_team_performance()

,MLB,NHL,NBA,NFL
MLB,NaN,0.000708,0.950540,0.802069
NHL,0.000708,NaN,0.022297,0.030883
NBA,0.950540,0.022297,NaN,0.941792
NFL,0.802069,0.030883,0.941792,NaN
